In [1]:
// Authenticate to applications insights.

// Get all the logs about this entity.

## Resources and Notes
- [Application insights API docs](https://dev.applicationinsights.io/documentation/Authorization/API-key-authentication)


format for call
https://api.applicationinsights.io/{api-version}/apps/{app-id}/{query-path}?[query-parameters]
 Headers "X-Api-Key: {api-key}"


vai's personal instance:
- API key: k7iqxklkgdikw6ybqlklfnmcu3air4cm7knr6581
- app ID: 912a76b4-a960-460e-b65d-30c8c7e181ca

In [1]:
using System.Net.Http;
using System.Text.Json;
using System.IO;

In [1]:
// Constants

public static class Constants{
    public const string applicationId = "912a76b4-a960-460e-b65d-30c8c7e181ca";  
    public const string apiKey = "k7iqxklkgdikw6ybqlklfnmcu3air4cm7knr6581"; 
    public const string host = "api.applicationinsights.io";
    public const string apiVersion = "v1";
    public const string query ="metrics/requests/count";
}

In [1]:
public static JsonString GetQuery()
{
    const string query = @"union customMetrics, customEvents | limit 30 | order  by  timestamp";
    var jsonstring = new {
        query = query
    };

    return jsonstring.SerializeToJson();
}
Console.WriteLine(GetQuery());


{"query":"union customMetrics, customEvents | limit 30 | order  by  timestamp"}


In [1]:
static async Task<JsonDocument?> QueryAppInsights()
{  
  try	
  {
    using HttpClient client = new HttpClient(); // typycally we would use only one http client per application.

    client.DefaultRequestHeaders.Add("X-Api-Key", Constants.apiKey);
      
    var uri = new UriBuilder();
    uri.Scheme = "https";
    uri.Host = Constants.host;
    uri.Path = $"{Constants.apiVersion}/apps/{Constants.applicationId}/query";
    Console.WriteLine( $"URL: {uri.ToString()}");

    var postContent = new StringContent(GetQuery().ToString(), Encoding.UTF8,  "application/json");
    
    var resp = client.PostAsync(uri.ToString(), postContent );
    await resp;
    // Console.WriteLine("REQUEST");
    // Console.WriteLine(resp.Result.RequestMessage);
    // Console.WriteLine(await resp.Result.RequestMessage.Content.ReadAsStringAsync());
    // Console.WriteLine("RESPONSE");
    // Console.WriteLine(resp.Result);
    // Console.WriteLine(await resp.Result.Content.ReadAsStringAsync());
    resp.Result.EnsureSuccessStatusCode();
    var responseStream = resp.Result.Content.ReadAsStream();
    var responseJson = await JsonDocument.ParseAsync(responseStream);
    return responseJson;
  }
  catch(HttpRequestException e)
  {
     Console.WriteLine("\nException Caught!");	
     Console.WriteLine("Message :{0} ",e.Message);
     return null;
  }
}


In [1]:
public static JsonDocument queryResult; 
queryResult = await QueryAppInsights();

URL: https://api.applicationinsights.io/v1/apps/912a76b4-a960-460e-b65d-30c8c7e181ca/query


In [1]:
Console.OutputEncoding = System.Text.Encoding.UTF8;

var toprint = queryResult.RootElement;

toprint


In [1]:
var items= queryResult.RootElement.GetProperty("tables")[0]
.GetProperty("columns")
.EnumerateArray()
.Select(i => i.GetProperty("name").ToString());


display(items);

index,value
0,timestamp
1,name
2,value
3,valueCount
4,valueSum
5,valueMin
6,valueMax
7,valueStdDev
8,customDimensions
9,operation_Name


In [1]:
var customDimensions = queryResult.RootElement.GetProperty("tables")[0]
.GetProperty("rows")
.EnumerateArray()
.Select(i => ( i[8].ToString(),  i[1].ToString()));

customDimensions
.Select( i=> JsonDocument.Parse(i.Item1))
.First()

RootElement
"{""target"":""15""}"
